In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import argparse
import os

import numpy as np
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from keras.layers import merge
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import list_pictures, array_to_img

from image_ext import list_pictures_in_multidir, load_imgs_asarray

np.random.seed(2016)

Using TensorFlow backend.


In [2]:
def create_fcn00(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv5)

    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)

    up7 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv5])
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv7)

    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv7), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', padding='same', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(inputs=inputs, outputs=conv12)

    return fcn

In [3]:
def create_fcn01(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)
    
    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', padding='same', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(inputs=inputs, outputs=conv12)

    return fcn

In [4]:
def create_fcn02(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv4), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', padding='same', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(inputs=inputs, outputs=conv12)

    return fcn

In [5]:
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2.*intersection + 1) / (K.sum(y_true) + K.sum(y_pred) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [6]:
def load_fnames(paths):
    f = open(paths)
    data1 = f.read()
    f.close()
    lines = data1.split('\n')
    #print(len(lines))
    # 最終行は空行なので消す
    del(lines[len(lines)-1])
    #print(len(lines))
    return lines

In [7]:
def make_fnames(fnames,fpath,fpath_mask,mask_ext):
    fnames_img = [];
    fnames_mask= [];
    
    for i in range(len(fnames)):
        fnames_img.append(fpath + '/' + fnames[i]);
        fnames_mask.append(fpath_mask + '/' + mask_ext + fnames[i]);
        
    return [fnames_img,fnames_mask]

In [8]:
if __name__ == '__main__':
    
    target_size = (224, 224)
    dpath_this = './'
    dname_checkpoints = 'checkpoints01'
    dname_outputs = 'outputs'
    fname_architecture = 'architecture.json'
    fname_weights = "model_weights_{epoch:02d}.h5"
    fname_stats = 'stats01.npz'
    dim_ordering = 'channels_first'
    
    # definision of mode, LEARN or TEST
    mode = "TEST"
    
    # モデルを作成
    print('creating model...')
    model = create_fcn02(target_size)
    model.summary()    

creating model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 224, 224)  896         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 32, 224, 224)  9248        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 32, 112, 112)  0           conv2d_2[0][0]                   
_________________________________________________________________________

In [9]:
    if mode == "LEARN":
        # Read Learning Data
        fnames = load_fnames('data/list_train_01.txt')
        [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data/img','data/mask','OperatorA_')

        X_train = load_imgs_asarray(fpaths_xs_train, grayscale=False, target_size=target_size,
                                    dim_ordering=dim_ordering)
        Y_train = load_imgs_asarray(fpaths_ys_train, grayscale=True, target_size=target_size,
                                    dim_ordering=dim_ordering) 

        # Read Validation Data
        fnames = load_fnames('data/list_valid_01.txt')
        [fpaths_xs_valid,fpaths_ys_valid] = make_fnames(fnames,'data/img','data/mask','OperatorA_')

        X_valid = load_imgs_asarray(fpaths_xs_valid, grayscale=False, target_size=target_size,
                                    dim_ordering=dim_ordering)
        Y_valid = load_imgs_asarray(fpaths_ys_valid, grayscale=True, target_size=target_size,
                                    dim_ordering=dim_ordering)     

In [10]:
        print('==> ' + str(len(X_train)) + ' training images loaded')
        print('==> ' + str(len(Y_train)) + ' training masks loaded')
        print('==> ' + str(len(X_valid)) + ' validation images loaded')
        print('==> ' + str(len(Y_valid)) + ' validation masks loaded')

NameError: name 'X_train' is not defined

In [11]:
        # 前処理
        print('computing mean and standard deviation...')
        mean = np.mean(X_train, axis=(0, 2, 3))
        std = np.std(X_train, axis=(0, 2, 3))
        print('==> mean: ' + str(mean))
        print('==> std : ' + str(std))

        print('saving mean and standard deviation to ' + fname_stats + '...')
        stats = {'mean': mean, 'std': std}
        np.savez(fname_stats, **stats)
        print('==> done')

        print('globally normalizing data...')
        for i in range(3):
            X_train[:, i] = (X_train[:, i] - mean[i]) / std[i]
            X_valid[:, i] = (X_valid[:, i] - mean[i]) / std[i]
        Y_train /= 255
        Y_valid /= 255
        print('==> done')

computing mean and standard deviation...


NameError: name 'X_train' is not defined

In [12]:
        # 損失関数，最適化手法を定義
        adam = Adam(lr=1e-5)
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.1, nesterov=True)
        #rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])

        # 構造・重みを保存するディレクトリーの有無を確認
        dpath_checkpoints = os.path.join(dpath_this, dname_checkpoints)
        if not os.path.isdir(dpath_checkpoints):
            os.mkdir(dpath_checkpoints)
        # モデルの構造を保存
        json_string = model.to_json()
        fpath_architecture = os.path.join(dpath_checkpoints, fname_architecture)
        with open(fpath_architecture, "w", encoding="utf8") as f:
            f.write(json_string)
        # 重みを保存するためのオブジェクトを用意
        fpath_weights = os.path.join(dpath_checkpoints, fname_weights)
        checkpointer = ModelCheckpoint(filepath=fpath_weights, save_best_only=False)        
        

In [13]:
        # トレーニングを開始
        print('start training...')
        history = model.fit(X_train, Y_train, batch_size=32, epochs=100, verbose=1,
                      shuffle=True, validation_data=(X_valid, Y_valid), callbacks=[checkpointer])

start training...


NameError: name 'X_train' is not defined

In [38]:
    if mode == "TEST":
        # Prediction (test) mode
        
        # 学習済みの重みをロード
        epoch = 99
        fname_weights = 'model_weights_%02d.h5'%(epoch)
        fpath_weights = os.path.join(dname_checkpoints, fname_weights)
        model.load_weights(fpath_weights)
        print('==> done')

==> done


In [44]:
        # Read Test Data
        fnames = load_fnames('data/list_test_01.txt')
        
        [fpaths_xs_test,fpaths_ys_test] = make_fnames(fnames,'data/img','data/mask','OperatorA_')

        X_test = load_imgs_asarray(fpaths_xs_test, grayscale=False, target_size=target_size,
                                    dim_ordering=dim_ordering)
        Y_test = load_imgs_asarray(fpaths_ys_test, grayscale=True, target_size=target_size,
                                    dim_ordering=dim_ordering)
        
            
        # トレーニング時に計算した平均・標準偏差をロード    
        print('loading mean and standard deviation from ' + fname_stats + '...')
        stats = np.load(fname_stats)
        mean = stats['mean']
        std = stats['std']
        print('==> mean: ' + str(mean))
        print('==> std : ' + str(std))
        
        for i in range(3):
            X_test[:, i] = (X_test[:, i] - mean[i]) / std[i]
        print('==> done')

loading mean and standard deviation from stats01.npz...
==> mean: [ 130.65464783   91.26850128   76.63642883]
==> std : [ 55.28170013  43.99096298  43.11348343]
==> done


In [45]:
        # テストを開始
        outputs = model.predict(X_test)

In [46]:
        print(outputs.shape)

(270, 1, 224, 224)


In [47]:
        # 出力を画像として保存
        dname_outputs = './outputs/'
        if not os.path.isdir(dname_outputs):
            print('create directory: %s'%(dname_outputs))
            os.mkdir(dname_outputs)

        print('saving outputs as images...')
        n = 0
        for i, array in enumerate(outputs):
            array = np.where(array > 0.5, 1, 0) # 二値に変換
            array = array.astype(np.float32)
            img_out = array_to_img(array, dim_ordering)
            # fpath_out = os.path.join(dname_outputs, fnames[i])
            fpath_out = os.path.join(dname_outputs, "%05d.jpg"%(n))
            img_out.save(fpath_out)
            n = n + 1
        
        print('==> done')

saving outputs as images...
==> done


In [51]:
    from PIL import Image
    import matplotlib.pyplot as plt

    n = 0
    for i in range(len(fpaths_xs_test)):
        # テスト画像
        im1 = Image.open(fpaths_xs_test[i])
        im1 = im1.resize((320,240)) 
        # 出力結果
        im2 = Image.open(os.path.join(dname_outputs, "%05d.jpg"%(n)))
        im2 = im2.resize((320,240))
        im2_d = np.zeros((im2.size[1],im2.size[0],3), 'uint8')
        im2_d[:,:,0] = np.array(im2)
        im2_d[:,:,1] = np.array(im2)
        im2_d[:,:,2] = np.array(im2)
        # Grond Truth
        im3 = Image.open(fpaths_ys_test[i])
        im3 = im2.resize((320,240))
        
        plt.imshow(np.hstack(np.hstack(np.array(im1),np.array(im2_d)),np.array(im3)))
        plt.show()
        
        n = n + 1

TypeError: hstack() takes 1 positional argument but 2 were given